In [ ]:
from scipy import misc, ndimage, signal
from sklearn.model_selection  import train_test_split
import numpy
import numpy as np
import random
import ntpath
import os
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from keras import optimizers 
from keras import regularizers
import tensorflow as tf
import cv2
from time import time
import time as tm
import datetime
from operator import itemgetter
import glob
from skimage.util.shape import view_as_blocks
from keras.utils import np_utils
from keras.utils import to_categorical

In [ ]:
!nvidia-smi
#!kill PIDnumber

In [ ]:
path_folder = "./"
path_img_base = './images'
if not os.path.exists(path_img_base):
    os.makedirs(path_img_base)

In [ ]:
################################################## 30 original SRM FILTERS
srm_weights = np.load(path_folder+'/SRM_Kernels1.npy') 
print (srm_weights.shape)
################################################## bias
biasSRM=numpy.ones(30)
################################################## TLU ACTIVATION FUNCTION
def Tanh3(x):
    tanh3 = tf.keras.activations.tanh(x)*3
    return tanh3
##################################################

In [ ]:
def GBRAS_Net2():
    tf.keras.backend.clear_session()
    #Inputs
    inputs = tf.keras.Input(shape=(256,256,1), name="input_1")
    #Layer 1
    layers_ty = tf.keras.layers.Conv2D(30, (5,5), weights=[srm_weights,biasSRM], strides=(1,1), padding='same', trainable=False, activation=Tanh3, use_bias=True)(inputs)
    layers_tn = tf.keras.layers.Conv2D(30, (5,5), weights=[srm_weights,biasSRM], strides=(1,1), padding='same', trainable=True, activation=Tanh3, use_bias=True)(inputs)
  
    layers1 = tf.keras.layers.add([layers_ty, layers_tn])
    layers1 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(layers1)
    #Layer 2
    layers = tf.keras.layers.DepthwiseConv2D(1)(layers1)
    layers = tf.keras.layers.SeparableConv2D(30,(3,3), padding='same', activation=None,depth_multiplier=3)(layers) 
    layers = tf.keras.layers.ReLU(negative_slope=0.1, threshold=0)(layers)
    layers = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(layers)
    #Layer 3
    layers = tf.keras.layers.DepthwiseConv2D(1)(layers)
    layers = tf.keras.layers.SeparableConv2D(30,(3,3), padding='same', activation=None,depth_multiplier=3)(layers) 
    layers = tf.keras.layers.ReLU(negative_slope=0.1, threshold=0)(layers)
    layers2 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(layers)
    skip1 =   tf.keras.layers.Add()([layers1, layers2])
    #Layer 4
    layers = tf.keras.layers.Conv2D(30, (3,3), strides=(1,1), activation=None, padding='same', kernel_initializer='glorot_uniform')(skip1) 
    layers = tf.keras.layers.ReLU(negative_slope=0.1, threshold=0)(layers)
    layers = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(layers)
    #Layer 5
    layers = tf.keras.layers.Conv2D(30, (3,3), strides=(1,1), activation=None, padding='same', kernel_initializer='glorot_uniform')(layers)
    layers = tf.keras.layers.ReLU(negative_slope=0.1, threshold=0)(layers)
    layers = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(layers)
    #Layer 6
    layers = tf.keras.layers.AveragePooling2D((2,2), strides= (2,2))(layers)
    #Layer 7
    layers = tf.keras.layers.Conv2D(60, (3,3), strides=(1,1), activation=None, padding='same', kernel_initializer='glorot_uniform')(layers) 
    layers = tf.keras.layers.ReLU(negative_slope=0.1, threshold=0)(layers)
    layers3 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(layers)
    #Layer 8
    layers = tf.keras.layers.DepthwiseConv2D(1)(layers3)
    layers = tf.keras.layers.SeparableConv2D(60,(3,3), padding='same', activation=None,depth_multiplier=3)(layers) 
    layers = tf.keras.layers.ReLU(negative_slope=0.1, threshold=0)(layers)
    layers = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(layers)
    #Layer 9
    layers = tf.keras.layers.DepthwiseConv2D(1)(layers)
    layers = tf.keras.layers.SeparableConv2D(60,(3,3), padding='same', activation=None,depth_multiplier=3)(layers) 
    layers = tf.keras.layers.ReLU(negative_slope=0.1, threshold=0)(layers)
    layers4 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(layers)
    skip2 =   tf.keras.layers.Add()([layers3, layers4])
    #Layer 10
    layers = tf.keras.layers.Conv2D(60, (3,3), strides=(1,1), activation=None, padding='same', kernel_initializer='glorot_uniform')(skip2) 
    layers = tf.keras.layers.ReLU(negative_slope=0.1, threshold=0)(layers)
    layers = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(layers)
    #Layer 11
    layers = tf.keras.layers.AveragePooling2D((2,2), strides= (2,2))(layers)
    #Layer 12
    layers = tf.keras.layers.Conv2D(60, (3,3), strides=(1,1), activation=None, padding='same', kernel_initializer='glorot_uniform')(layers) 
    layers = tf.keras.layers.ReLU(negative_slope=0.1, threshold=0)(layers)
    layers = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(layers)
    #Layer 13
    layers = tf.keras.layers.AveragePooling2D((2,2), strides= (2,2))(layers)
    #Layer 14
    layers = tf.keras.layers.Conv2D(60, (3,3), strides=(1,1), activation=None, padding='same', kernel_initializer='glorot_uniform')(layers) 
    layers = tf.keras.layers.ReLU(negative_slope=0.1, threshold=0)(layers)
    layers = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(layers)
    #Layer 15
    layers = tf.keras.layers.AveragePooling2D((2,2), strides= (2,2))(layers)
    #Layer 16
    layers = tf.keras.layers.Conv2D(30, (1,1), strides=(1,1), activation=None, padding='same', kernel_initializer='glorot_uniform')(layers) 
    layers = tf.keras.layers.ReLU(negative_slope=0.1, threshold=0)(layers)
    layers = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(layers)
    #Layer 17
    layers = tf.keras.layers.Conv2D(2, (1,1), strides=(1,1), activation=None, padding='same', kernel_initializer='glorot_uniform')(layers) 
    layers = tf.keras.layers.ReLU(negative_slope=0.1, threshold=0)(layers)
    layers = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(layers)
    #Layer 18
    layers = tf.keras.layers.GlobalAveragePooling2D(data_format="channels_last")(layers)
    #Layer 19
    predictions = tf.keras.layers.Softmax(axis=1)(layers)
    #Model generation
    model = tf.keras.Model(inputs = inputs, outputs=predictions)
    #Optimizer
    optimizer=tf.keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    print ("Model GBRAS-Net Generated")
    #Model compilation
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
model_untrained = GBRAS_Net2()

In [ ]:
path_model_trained2 = path_folder+"/GBRAS_WOW04_BIFURCATION.hdf5"
model_trained2 = tf.keras.models.load_model(path_model_trained2, custom_objects={'Tanh3':Tanh3}, compile=True)

In [ ]:
path_stego = "/media/ia/Datos1/DocReinel/Steganalysis/H_Brayan_A_Arteaga/Frankenstein_Project/GBRAS-Net/DATABASES/BOSSbase-1.01/WOW/0.4bpp/stego"
path_cover = "/media/ia/Datos1/DocReinel/Steganalysis/H_Brayan_A_Arteaga/Frankenstein_Project/GBRAS-Net/DATABASES/BOSSbase-1.01/cover"

n=256
def load_images(path_pattern):
    files=glob.glob(path_pattern)
    X=[]
    for f in sorted(files):
        I = cv2.imread(f, cv2.IMREAD_GRAYSCALE)
        patches = view_as_blocks(I, (n, n))
        for i in range(patches.shape[0]):
            for j in range(patches.shape[1]):
                X.append( [ patches[i,j] ] )
    X=numpy.array(X)
    return X

Xc_ = load_images(path_cover+'/*.pgm') ##COVER IMAGES
Xs_ = load_images(path_stego+'/*.pgm') ##STEGO IMAGES
X_  = (numpy.vstack((Xc_, Xs_)))
Xt_ = (numpy.hstack(([0]*len(Xc_), [1]*len(Xs_))))
Xt_ = np_utils.to_categorical(Xt_, 2)
X_  = np.rollaxis(X_,1,4)  #channel axis shifted to last axis

print("Total image data and labels",X_.shape,Xt_.shape)
#Cover hasta las 10000 ##Train hasta las 4000 ##Valid hasta de las 4000 a las 5000 ##Test de las 5000 a las 10000
X_train = np.concatenate([X_[0:4000],X_[10000:14000]],axis=0)
X_valid = np.concatenate([X_[4000:5000],X_[14000:15000]],axis=0)
X_test  = np.concatenate([X_[5000:10000],X_[15000:20000]],axis=0)
y_train = np.concatenate([Xt_[0:4000],Xt_[10000:14000]],axis=0)
y_valid = np.concatenate([Xt_[4000:5000],Xt_[14000:15000]],axis=0)
y_test  = np.concatenate([Xt_[5000:10000],Xt_[15000:20000]],axis=0)
#Controled randomized data for training
X_dat0, X_dat1, y_dat0, y_dat1 = train_test_split(X_train, y_train, test_size=0.50, random_state=64) 
X_train = np.concatenate([X_dat0,X_dat1],axis=0) 
y_train = np.concatenate([y_dat0,y_dat1],axis=0) 
print(X_train.shape)
print(y_train.shape)
print(X_valid.shape)
print(y_valid.shape)
print(X_test.shape)
print(y_test.shape)


In [ ]:
loss,accuracy = model_untrained.evaluate(X_test,y_test,verbose=0) #Test BOSSbase 1.01 
print(f'Loss={loss:.4f} and Accuracy={accuracy:0.3f}')

loss,accuracy = model_trained2.evaluate(X_test,y_test,verbose=0) #Test BOSSbase 1.01 
print(f'Loss={loss:.4f} and Accuracy={accuracy:0.3f}')

In [ ]:
model = model_trained2

In [ ]:
#COVER COMO COVER
for i in range(5000):
    image = X_test[i,:,:,0]
    image = np.reshape(image,(1,256,256,1))  
    prediction1 = np.round(model.predict(image)) #cover
    if (prediction1 == np.array([1, 0])).all():     #cover as cover
        print(i)

In [ ]:
#STEGO COMO STEGO
for i in range(5000):
    image = X_test[5000+i,:,:,0]
    image = np.reshape(image,(1,256,256,1))  
    prediction2 = np.round(model.predict(image)) #stego 
    if (prediction2 == np.array([0, 1])).all():     #stego as stego
        print(i)

In [ ]:
#COVER COMO STEGO
for i in range(5000):
    image = X_test[i,:,:,0]
    image = np.reshape(image,(1,256,256,1))  
    prediction1 = np.round(model.predict(image)) #cover
    if (prediction1 == np.array([0, 1])).all():     #cover as stego
        print(i)

In [ ]:
#STEGO COMO COVER
for i in range(5000):
    image = X_test[5000+i,:,:,0]
    image = np.reshape(image,(1,256,256,1))  
    prediction2 = np.round(model.predict(image)) #stego
    if (prediction2 == np.array([1, 0])).all(): #stego as cover
        print(i)

In [ ]:
def cover_or_stego(image):
    plt.figure(figsize=(10,10))
    plt.imshow(image,cmap='gray')
    plt.title("Image cover or stego?")
    plt.show()
    image = np.reshape(image,(1,256,256,1))  
    prediction = np.round(model.predict(image))
    print(prediction)
    if (prediction == np.array([1, 0])).all():
        print("Prediction: COVER")
    else:
        print("Prediction: STEGO") 

In [ ]:
number_image = 16

#COVER data
image = X_test[number_image,:,:,0]
cover_or_stego(image)

In [ ]:
number_image = 193

#STEGO data
image = X_test[number_image+5000,:,:,0]
cover_or_stego(image)

In [ ]:
for i, layer in enumerate(model_trained2.layers): 
    print(i, layer.name) 

In [ ]:
# summarize filter shapes
for layer in model_trained2.layers:
    # check for convolutional layer
    if 'conv' not in layer.name:
        continue
    if 'separable_conv2d' in layer.name: 
        continue
    # get filter weights
    filters, biases = layer.get_weights()
    print(layer.name, filters.shape) 

In [ ]:
model.summary()

In [ ]:
def visualize_conv_layer_1x2(num,layer_name,image,row=1,col=2):
    if not os.path.exists(path_images+'/'+str(num)):
        os.makedirs(path_images+'/'+str(num))
    
    plt.figure(figsize=(50,50))
    plt.imshow(image,cmap="gray")
    plt.axis("off")
    plt.savefig(path_images+'/'+str(num)+'/'+'image'+layer_name+'.eps', format='eps')
    plt.savefig(path_images+'/'+str(num)+'/'+'image'+layer_name+'.svg', format='svg')
    plt.savefig(path_images+'/'+str(num)+'/'+'image'+layer_name+'.pdf', format='pdf')  
    plt.show()
    
    plt.figure()
    layer_output=model.get_layer(layer_name).output
    intermediate_model=tf.keras.models.Model(inputs=model.input,outputs=layer_output)
    image=numpy.reshape(image,(1,256,256,1))
    intermediate_prediction=intermediate_model.predict(image)
    print('Shape activations maps:',intermediate_prediction.shape)

    fig, (ax1, ax2) = plt.subplots(row, col, figsize=(30,30))
    fig.figsize=(2500,2500)
    ax1.imshow(intermediate_prediction[0, :, :, 0], cmap='gray')
    ax1.axis("off")
    ax2.imshow(intermediate_prediction[0, :, :, 1], cmap='gray')
    ax2.axis("off")

    plt.savefig(path_images+'/'+str(num)+'/'+'ACT-MAP_All_'+layer_name+'.eps', format='eps')
    plt.savefig(path_images+'/'+str(num)+'/'+'ACT-MAP_All_'+layer_name+'.svg', format='svg')
    plt.savefig(path_images+'/'+str(num)+'/'+'ACT-MAP_All_'+layer_name+'.pdf', format='pdf')  
    plt.show()

In [ ]:
model = model_trained2

In [ ]:
#Gráficas de cover como cover
images = [71,72,75,76,77,78,79,80,81,82]

### COVER IMAGE
path_images = './images/tp_tn_fp_fn/cover_cover'
if not os.path.exists(path_images):
    os.makedirs(path_images)

for image in images:
    img = X_test[image]
    visualize_conv_layer_1x2(int(image),'conv2d_9',img,1,2)
    

In [ ]:
#Gráficas de cover como stego
images = [812,814,815,831,851,852,855,856,860,861]

### COVER IMAGE
path_images = './images/tp_tn_fp_fn/cover_stego'
if not os.path.exists(path_images):
    os.makedirs(path_images)

for image in images:
    img = X_test[image]
    visualize_conv_layer_1x2(int(image),'conv2d_9',img,1,2)
    

In [ ]:
#Gráficas de stego como stego
images = [3,12,18,33,39,48,96,107,122,165]

### COVER IMAGE
path_images = './images/tp_tn_fp_fn/stego_stego'
if not os.path.exists(path_images):
    os.makedirs(path_images)

for image in images:
    img = X_test[image+5000]
    visualize_conv_layer_1x2(int(image+5000),'conv2d_9',img,1,2)
    

In [ ]:
#Gráficas de stego como cover
images = [5,14,17,19,20,32,54,70,71,100]

### COVER IMAGE
path_images = './images/tp_tn_fp_fn/stego_cover'
if not os.path.exists(path_images):
    os.makedirs(path_images)

for image in images:
    img = X_test[image+5000]
    visualize_conv_layer_1x2(int(image+5000),'conv2d_9',img,1,2)
    